In [1]:
#!pip install climetlab
#!pip install climetlab-maelstrom-radiation

In [2]:
import climetlab as cml
import numpy as np 
import matplotlib.pyplot as plt

# Radiation emulation: 3d correction.

This notebook is a sequel to demo_radiation. Please read through demo_radiation before continuing.

### The task
Existing models of radiation used in operational forecasting typically do not take into account of the 3D cloud aspect within a grid column. The ecRAD SPARTACUS (Hogan & Shonk 2012) solver takes such features into account. However this scheme is considerably more expensive than schemes such as McICA or Tripleclouds. In this task we seek to learn the difference between ecRAD predictions using the Tripleclouds solver and those using the SPARTACUS solver. 
A successful emulator of this dataset could be used as a corrective term to the Tripleclouds formulation, thereby incorporating the 3D cloud effects. By only seeking to learn the correction term, rather than the full output of the SPARTACUS solver we seek to only learn the most expensive calculations within radiative heating.

Using the argument `dataset = 3dcorrection` we can get the difference between the outputs of SPARTACUS and Tripleclouds.


# Loading the data
As with the previous notebook data can be loaded by either specifying a **subset**, e.g. 'tier-1' or by specifying the trio of **date, timestep, patch**. 

In addition to those, we need to prescribe the **dataset** to be **3dcorrection**.

### Subset
Currently only 'tier-1' is supported for the 3dcorrection problem. 
### Date/timestep/patch
These describe the start **date** of the numerical forecast used to generate the data, the **timestep** index of said forecast (note the time increment is 12 minutes, meaning step 125 corresponds to 25 hours after initialisation) and **patch** a spatial subset of globe (here the globe is divided into 16 equal regions).
Valid values for each of the above can be found with **cmlds.valid_date** etc.
Most start dates are from 2020, which we recommend as the training set, four start dates in 2019 are provided as validation/testing data.

Currently only the first date from 2020 is uploaded. More data to come ...

### If no descriptions are passed then tier-1 subset is loaded, corresponding to 2020-01-01/0/range(0,16,2)

In [7]:
cmlds = cml.load_dataset(
        'maelstrom-radiation',
        dataset='3dcorrection',
        subset='tier-1',
        raw_inputs=False,
    )

ds = cmlds.to_xarray()

Loading subset: tier-1
Loading date: 20200101, timestep: 0, patch: [0, 2, 4, 6, 8, 10, 12, 14]


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
print(ds)

<xarray.Dataset>
Dimensions:       (column: 135680, sca_variable: 17, level: 137, col_variable: 27, half_level: 138, hl_variable: 2, p_variable: 1, level_interface: 136, inter_variable: 1)
Dimensions without coordinates: column, sca_variable, level, col_variable, half_level, hl_variable, p_variable, level_interface, inter_variable
Data variables:
    sca_inputs    (column, sca_variable) float32 dask.array<chunksize=(16960, 1), meta=np.ndarray>
    col_inputs    (column, level, col_variable) float32 dask.array<chunksize=(16960, 137, 1), meta=np.ndarray>
    hl_inputs     (column, half_level, hl_variable) float32 dask.array<chunksize=(16960, 138, 1), meta=np.ndarray>
    pressure_hl   (column, half_level, p_variable) float32 dask.array<chunksize=(16960, 138, 1), meta=np.ndarray>
    inter_inputs  (column, level_interface, inter_variable) float32 dask.array<chunksize=(16960, 136, 1), meta=np.ndarray>
    flux_dn_sw    (column, half_level) float32 dask.array<chunksize=(16960, 138), meta=np

The inputs and outputs for the 3dcorrection problem are the same as those for the McICA or Tripleclouds problem. Please see the previous notebook for a discussion of the inputs and outputs.

Things to note. I recommend using
**raw_inputs = false** to concatenate similar shaped variables together.

### Predictors
The direct outputs from any of the radiation schemes are the upwards and downwards fluxes for both shortwave and longwave heating. These are described on 138 "half-levels", which are the boundaries to the 137 model levels.
The rest of the IFS model uses heating rates and fluxes at the boundaries (top and bottom) increment the temperatures. The heating rate is the vertical (pressure) derivative of the net (down - up) flux and is described on the 137 model levels.
This means that there are several modelling approaches, to be used separately on the shortwave and longwave.

- Predict only the fluxes. Assume that the derived heating will be accurate by producing accurate fluxes. This has been previously tried, but there were unrealistic gradients in the heating rate which degraded coupled predictions.
- Predict the fluxes, use the custom layer below to deduce the heating rates, and use a loss which combines the mse of the fluxes and the mse of the heating rates. NB the magnitude of the heating rates is much smaller than those of the fluxes, so the inbalance of the two terms will need to be large or the heating rates will need to be rescaled.
- Directly predict the heating rates and 4 boundary fluxes (down and up at top and bottom). NB downwards flux at the top of the atmosphere is equal to the solar_irradiance\*cos_solar_zenith_angle for shortwave and 0 for longwave. This approach can induce energy imbalances, which can be solved with an alternate formulation, which we ignore for simplicity here.



In [5]:
import tensorflow as tf

@tf.keras.utils.register_keras_serializable()
class HRLayer(tf.keras.layers.Layer):
    """
    Layer to calculate heating rates given fluxes
    and half-level pressures.
    This could be used to deduce the heating rates
    within the model so that the outputs can be 
    constrained by both fluxes and heating rates
    """
    def __init__(self,name=None,**kwargs):
        super(HRLayer, self).__init__(name=name,**kwargs)
        self.g_cp = tf.constant(9.80665 / 1004)
        
    def build(self, input_shape):
        pass

    def call(self, inputs):
        fluxes_down = inputs[0] # shape batch,138,1
        fluxes_down = inputs[1] # shape batch,138,1
        hlpress = inputs[2] # shape batch,138,1
        netflux = fluxes_down[...,0] - fluxes_up[...,0]
        flux_diff = netflux[...,1:] - netflux[...,:-1]
        net_press = hlpress[...,1:,0] - hlpress[...,:-1,0]
        return -self.g_cp * tf.math.divide(flux_diff,net_press)


# Machine learning this dataset
A TFRecord format of this dataset has not yet been generate (work in progress). Work is also ongoing to create a `to_tfdataset` function for this dataset.

# Questions?
Please reach out if you have questions/problems on this dataset.